# K8S Resource Test


In [1]:
SCHEDULER_IP=!kubectl get svc seldon-scheduler -n seldon-mesh -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
SCHEDULER_IP=SCHEDULER_IP[0]
import os
os.environ['SCHEDULER_IP'] = SCHEDULER_IP

In [2]:
MESH_IP=!kubectl get svc seldon-mesh -n seldon-mesh -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MESH_IP=MESH_IP[0]
import os
os.environ['MESH_IP'] = MESH_IP

## Show ServerConfigurations Available

In [3]:
!kubectl get serverconfig -n seldon-mesh

NAME       AGE
mlserver   68m
triton     68m


In [4]:
!kubectl get serverconfig -n seldon-mesh mlserver -o jsonpath='{.spec}' | jq .

{
  "podSpec": {
    "containers": [
      {
        "image": "seldonio/seldon-rclone:latest",
        "imagePullPolicy": "IfNotPresent",
        "name": "rclone",
        "ports": [
          {
            "containerPort": 5572,
            "name": "rclone",
            "protocol": "TCP"
          }
        ],
        "readinessProbe": {
          "failureThreshold": 3,
          "initialDelaySeconds": 5,
          "periodSeconds": 5,
          "successThreshold": 1,
          "tcpSocket": {
            "port": 5572
          },
          "timeoutSeconds": 1
        },
        "volumeMounts": [
          {
            "mountPath": "/mnt/agent",
            "name": "mlserver-models"
          }
        ]
      },
      {
        "env": [
          {
            "name": "SELDON_SERVER_CAPABILITIES",
            "value": "sklearn,xgboost"
          },
          {
            "name": "SELDON_OVERCOMMIT",
            "value": "false"
          },
          {
            "name": "SELDON_SER

## Deploy MLServer Instance

In [7]:
!cat ../../operator/samples/servers/mlserver1.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Server
metadata:
  name: mlserver1
  namespace: seldon-mesh
spec:
  server:
    config: mlserver


In [8]:
!kubectl create -f ../../operator/samples/servers/mlserver1.yaml

server.mlops.seldon.io/mlserver1 created


In [9]:
!kubectl wait --for condition=ready --timeout=300s server --all -n seldon-mesh

server.mlops.seldon.io/mlserver1 condition met


## Deploy Model

In [10]:
!cat ../../operator/samples/models/sklearn-iris-gs.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: iris
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/mlserver/iris"
  artifactVersion: 1
  requirements:
  - sklearn
  replicas: 1


In [11]:
!kubectl create -f ../../operator/samples/models/sklearn-iris-gs.yaml

model.mlops.seldon.io/iris created


In [12]:
!kubectl wait --for condition=ready --timeout=300s model --all -n seldon-mesh

model.mlops.seldon.io/iris condition met


In [13]:
!kubectl get model iris -n seldon-mesh -o jsonpath='{.status}' | jq .

{
  "conditions": [
    {
      "lastTransitionTime": "2022-01-27T18:39:12Z",
      "status": "True",
      "type": "ModelReady"
    },
    {
      "lastTransitionTime": "2022-01-27T18:39:12Z",
      "status": "True",
      "type": "Ready"
    }
  ]
}


In [14]:
!curl -v http://${MESH_IP}/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

* Expire in 0 ms for 6 (transfer 0x5565af3ea4f0)
*   Trying 172.18.255.3...
* TCP_NODELAY set
* Expire in 200 ms for 4 (transfer 0x5565af3ea4f0)
* Connected to 172.18.255.3 (172.18.255.3) port 80 (#0)
> POST /v2/models/iris/infer HTTP/1.1
> Host: 172.18.255.3
> User-Agent: curl/7.64.0
> Accept: */*
> Content-Type: application/json
> seldon-model: iris
> Content-Length: 94
> 
* upload completely sent off: 94 out of 94 bytes
< HTTP/1.1 200 OK
< date: Thu, 27 Jan 2022 18:39:23 GMT
< server: envoy
< content-length: 194
< content-type: application/json
< x-envoy-upstream-service-time: 1424
< 
* Connection #0 to host 172.18.255.3 left intact
{"model_name":"iris","model_version":"1","id":"e4ce6590-13c5-4ba3-9ace-8a2ca2fb6096","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"INT64","parameters":null,"data":[2]}]}

In [15]:
!kubectl get server mlserver1 -n seldon-mesh -o jsonpath='{.status}' | jq .

{
  "conditions": [
    {
      "lastTransitionTime": "2022-01-27T18:39:01Z",
      "status": "True",
      "type": "Ready"
    },
    {
      "lastTransitionTime": "2022-01-27T18:39:01Z",
      "reason": "StatefulSet replicas matches desired replicas",
      "status": "True",
      "type": "StatefulSetReady"
    }
  ],
  "loadedModels": 1
}


## Deploy Second Server

In [16]:
!cat ../../operator/samples/servers/mlserver2.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Server
metadata:
  name: mlserver2
  namespace: seldon-mesh
spec:
  server:
    config: mlserver


In [17]:
!kubectl create -f ../../operator/samples/servers/mlserver2.yaml

server.mlops.seldon.io/mlserver2 created


In [18]:
!kubectl wait --for condition=ready --timeout=300s server --all -n seldon-mesh

server.mlops.seldon.io/mlserver1 condition met
server.mlops.seldon.io/mlserver2 condition met


## Delete First Server

In [19]:
!kubectl delete -f ../../operator/samples/servers/mlserver1.yaml

server.mlops.seldon.io "mlserver1" deleted


In [20]:
!kubectl get server mlserver2 -n seldon-mesh -o jsonpath='{.status}' | jq .

{
  "conditions": [
    {
      "lastTransitionTime": "2022-01-27T18:40:04Z",
      "status": "True",
      "type": "Ready"
    },
    {
      "lastTransitionTime": "2022-01-27T18:40:04Z",
      "reason": "StatefulSet replicas matches desired replicas",
      "status": "True",
      "type": "StatefulSetReady"
    }
  ],
  "loadedModels": 1
}


In [21]:
!curl -v http://${MESH_IP}/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

* Expire in 0 ms for 6 (transfer 0x5592d63534f0)
*   Trying 172.18.255.3...
* TCP_NODELAY set
* Expire in 200 ms for 4 (transfer 0x5592d63534f0)
* Connected to 172.18.255.3 (172.18.255.3) port 80 (#0)
> POST /v2/models/iris/infer HTTP/1.1
> Host: 172.18.255.3
> User-Agent: curl/7.64.0
> Accept: */*
> Content-Type: application/json
> seldon-model: iris
> Content-Length: 94
> 
* upload completely sent off: 94 out of 94 bytes
< HTTP/1.1 200 OK
< date: Thu, 27 Jan 2022 18:40:50 GMT
< server: envoy
< content-length: 194
< content-type: application/json
< x-envoy-upstream-service-time: 1441
< 
* Connection #0 to host 172.18.255.3 left intact
{"model_name":"iris","model_version":"2","id":"977eb13a-80e2-42a9-8369-db2de8b9e1c3","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"INT64","parameters":null,"data":[2]}]}

In [22]:
!kubectl delete -f ../../operator/samples/models/sklearn-iris-gs.yaml

model.mlops.seldon.io "iris" deleted


In [23]:
!kubectl get server mlserver2 -n seldon-mesh -o jsonpath='{.status}' | jq .

{
  "conditions": [
    {
      "lastTransitionTime": "2022-01-27T18:40:04Z",
      "status": "True",
      "type": "Ready"
    },
    {
      "lastTransitionTime": "2022-01-27T18:40:04Z",
      "reason": "StatefulSet replicas matches desired replicas",
      "status": "True",
      "type": "StatefulSetReady"
    }
  ],
  "loadedModels": 0
}


In [24]:
!kubectl delete -f ../../operator/samples/servers/mlserver2.yaml

server.mlops.seldon.io "mlserver2" deleted
